In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
pd.options.display.max_columns = 1000

train = pd.read_csv("/kaggle/input/home-credit-default-risk/application_train.csv")
test = pd.read_csv("/kaggle/input/home-credit-default-risk/application_test.csv")
display(train,test)

In [ ]:
from sklearn.model_selection import train_test_split, StratifiedKFold
train, _ = train_test_split(train, test_size=0.75,random_state=42,stratify=train["TARGET"])

In [ ]:
ip = pd.read_csv("/kaggle/input/home-credit-default-risk/installments_payments.csv")
# ip 거의 다 숫자칼럼.

In [ ]:
ip_num = ip.drop(["SK_ID_PREV"],1).groupby("SK_ID_CURR").agg(["mean","min",'max','std','median','count'])

In [ ]:
ccb = pd.read_csv("/kaggle/input/home-credit-default-risk/credit_card_balance.csv")
# ccb #이것도 거의 다 숫자칼럼.

In [ ]:
ccb_num = ccb.drop(["SK_ID_PREV"],1).groupby("SK_ID_CURR").agg(["mean","min",'max','std','median','count'])

In [ ]:
ccb_cat = ccb.groupby("SK_ID_CURR")[ccb.columns[ccb.dtypes == object]].nunique()

In [ ]:
ccb_onehot = pd.concat([ccb["SK_ID_CURR"], pd.get_dummies(ccb[ccb.columns[ccb.dtypes == object]])], 1).groupby("SK_ID_CURR").sum()

In [ ]:
pa = pd.read_csv("/kaggle/input/home-credit-default-risk/previous_application.csv")

In [ ]:
pa_num = pa.drop(["SK_ID_PREV"],1).groupby("SK_ID_CURR").agg(["mean","min",'max','std','median','count'])

In [ ]:
pa_cat = pa.groupby("SK_ID_CURR")[pa.columns[pa.dtypes == object]].nunique()

In [ ]:
pa_onehot = pd.concat([pa["SK_ID_CURR"], pd.get_dummies(pa[pa.columns[pa.dtypes == object]])], 1).groupby("SK_ID_CURR").sum()

In [ ]:
pcb = pd.read_csv("/kaggle/input/home-credit-default-risk/POS_CASH_balance.csv")

In [ ]:
pcb_num = pcb.drop(["SK_ID_PREV"],1).groupby("SK_ID_CURR").agg(["mean","min",'max','std','median','count'])

In [ ]:
pcb_cat = pcb.groupby("SK_ID_CURR")[pcb.columns[pcb.dtypes == object]].nunique()

In [ ]:
pcb_onehot = pd.concat([pcb["SK_ID_CURR"], pd.get_dummies(pcb[pcb.columns[pcb.dtypes == object]])], 1).groupby("SK_ID_CURR").sum()

In [ ]:
b = pd.read_csv("/kaggle/input/home-credit-default-risk/bureau.csv")
bb = pd.read_csv("/kaggle/input/home-credit-default-risk/bureau_balance.csv")

In [ ]:
bb_num = bb.groupby("SK_ID_BUREAU").agg(["mean","min",'max','std','median','count'])


In [ ]:
b = pd.merge(b, bb_num, how="left", on="SK_ID_BUREAU")

In [ ]:
b_num = b.drop(["SK_ID_BUREAU"],1).groupby("SK_ID_CURR").agg(["mean","min",'max','std','median','count'])


In [ ]:
b_cat = b.groupby("SK_ID_CURR")[b.columns[b.dtypes == object]].nunique()

In [ ]:
b_onehot = pd.concat([b["SK_ID_CURR"], pd.get_dummies(b[b.columns[b.dtypes == object]])], 1).groupby("SK_ID_CURR").sum()

In [ ]:
all_data = pd.concat([train,test])

In [ ]:
all_data = pd.merge(all_data, ip_num, how="left", on="SK_ID_CURR")

In [ ]:
all_data = pd.merge(all_data, ccb_num, how="left", on="SK_ID_CURR")
all_data = pd.merge(all_data, ccb_cat, how="left", on="SK_ID_CURR")
all_data = pd.merge(all_data, ccb_onehot, how="left", on="SK_ID_CURR")

In [ ]:
all_data = pd.merge(all_data, pa_num, how="left", on ="SK_ID_CURR")
all_data = pd.merge(all_data, pa_cat, how="left", on ="SK_ID_CURR")
all_data = pd.merge(all_data, pa_onehot, how="left", on ="SK_ID_CURR")

In [ ]:
all_data = pd.merge(all_data, pcb_num, how="left", on ="SK_ID_CURR")
all_data = pd.merge(all_data, pcb_cat, how="left", on ="SK_ID_CURR")
all_data = pd.merge(all_data, pcb_onehot, how="left", on ="SK_ID_CURR")

In [ ]:
all_data = pd.merge(all_data, b_num, how="left", on ="SK_ID_CURR")
all_data = pd.merge(all_data, b_cat, how="left", on ="SK_ID_CURR")
all_data = pd.merge(all_data, b_onehot, how="left", on ="SK_ID_CURR")

In [ ]:
pd.options.display.max_rows = 1000

In [ ]:
all_data.head(100)

In [ ]:
all_data2 = all_data.drop(["TARGET","SK_ID_CURR"],1)

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
cate_cols = all_data2.columns[all_data2.dtypes == 'object']
for i in cate_cols:
    all_data2[i] = le.fit_transform(list(all_data2[i]))

In [ ]:
all_data["DAYS_EMPLOYED"].value_counts()

In [ ]:
(train == 365243).sum()

In [ ]:
# 결측치처리
all_data2 = all_data2.replace(365243, np.nan)
all_data2 = all_data2.fillna(-1)

In [ ]:
all_data2["AMT_IT/C"] = all_data2["AMT_INCOME_TOTAL"] / all_data2["AMT_CREDIT"]
all_data2["AMT_IT/A"] = all_data2["AMT_INCOME_TOTAL"] / all_data2["AMT_ANNUITY_x"]
all_data2["AMT_IT/DE"] = all_data2["AMT_INCOME_TOTAL"] / all_data2["DAYS_EMPLOYED"]
all_data2["AMT_IT/DB"] = all_data2["AMT_INCOME_TOTAL"] / all_data2["DAYS_BIRTH"]
all_data2["AMT_IT/CFM"] = all_data2["AMT_INCOME_TOTAL"] / all_data2["CNT_FAM_MEMBERS"]

all_data2["AMT_A/C"] = all_data2["AMT_ANNUITY_x"] / all_data2["AMT_CREDIT"]

all_data2["DAYS_BIRTH/DE"] = all_data2["DAYS_BIRTH"] / all_data2["DAYS_EMPLOYED"]
all_data2["OWN_CAR_AGE/DE"] = all_data2["OWN_CAR_AGE"] / all_data2["DAYS_EMPLOYED"]
all_data2["OWN_CAR_AGE/DB"] = all_data2["OWN_CAR_AGE"] / all_data2["DAYS_BIRTH"]

In [ ]:
train2 = all_data2[:len(train)]
test2 = all_data2[len(train):]

In [ ]:
from catboost import CatBoostClassifier
cb = CatBoostClassifier()
cb.fit(train2, train['TARGET'])
result = cb.predict_proba(test2)

In [ ]:
sub = pd.read_csv("/kaggle/input/home-credit-default-risk/sample_submission.csv")
sub["TARGET"] = result[:,1]
sub.to_csv("subsub.csv", index=0)